In [178]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error  # 루트 제곱 평균 오차
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.preprocessing import StandardScaler  # 평균 0, 표준편차 1
from sklearn.preprocessing import MinMaxScaler  # 백분위
from sklearn.preprocessing import RobustScaler

In [9]:
import sklearn

# dir(sklearn.naive_bayes)

In [10]:
file_url = (
    "https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv"
)
data = pd.read_csv(file_url)
data.shape

(8128, 13)

In [12]:
data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [15]:
data.describe(include="all")

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
count,8128,8128.000000,8.128000e+03,8.128000e+03,8128,8128,8128,8128,7907,7907,7913,7906,7907.000000
unique,2058,NaN,NaN,NaN,4,3,2,5,393,121,322,441,NaN
top,Maruti Swift Dzire VDI,NaN,NaN,NaN,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,NaN
freq,129,NaN,NaN,NaN,4402,6766,7078,5289,225,1017,377,530,NaN
mean,NaN,2013.804011,6.382718e+05,6.981951e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.416719
std,NaN,4.044249,8.062534e+05,5.655055e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.959588
min,NaN,1983.000000,2.999900e+04,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000
25%,NaN,2011.000000,2.549990e+05,3.500000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000
50%,NaN,2015.000000,4.500000e+05,6.000000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000
75%,NaN,2017.000000,6.750000e+05,9.800000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000


# 전처리 : 텍스트 데이터

In [22]:
# data["engine"].str.replace("CC", "").str.replace(" ", "")[0]
# data["engine"].str.replace(" CC", "")[0]
data["engine"].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [23]:
data[["engine", "engine_unit"]] = data["engine"].str.split(expand=True)
data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,engine_unit
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248,74 bhp,190Nm@ 2000rpm,5.0,CC
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498,103.52 bhp,250Nm@ 1500-2500rpm,5.0,CC


In [25]:
data["engine"] = data["engine"].astype("float32")

In [27]:
data["engine"].dtype

dtype('float32')

In [28]:
data["engine_unit"].unique()

array(['CC', nan], dtype=object)

In [30]:
data["engine"].isna().sum()

np.int64(221)

In [31]:
data.drop("engine_unit", axis=1, inplace=True)

In [32]:
data[["max_power", "max_power_unit"]] = data["max_power"].str.split(expand=True)
data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74,190Nm@ 2000rpm,5.0,bhp
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52,250Nm@ 1500-2500rpm,5.0,bhp


In [36]:
data["max_power"] = data["max_power"].astype("float32")

ValueError: could not convert string to float: 'bhp'

In [37]:
data[data["max_power"] == "bhp"]

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [44]:
type(pd.NA)
import numpy as np
type(np.nan)

float

In [46]:
import numpy as np

def isFloat(value):
  try:
    #정상일때실행
    float(value)
    return float(value)
  except ValueError:
    #에러일때실행
    return np.nan

In [47]:
data["max_power"] = data["max_power"].apply(isFloat)

In [49]:
data["max_power"] = data["max_power"].astype("float32")

In [50]:
data["max_power_unit"].unique()

array(['bhp', nan, None], dtype=object)

In [51]:
data.drop("max_power_unit", axis=1, inplace=True)

In [55]:
data[["mileage", "mileage_unit"]] = data["mileage"].str.split(expand=True)
data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,mileage_unit
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4,1248.0,74.000000,190Nm@ 2000rpm,5.0,kmpl
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.519997,250Nm@ 1500-2500rpm,5.0,kmpl


In [56]:
data["mileage"] = data["mileage"].astype("float32")

In [57]:
data["mileage_unit"].unique()

array(['kmpl', 'km/kg', nan], dtype=object)

In [58]:
data["fuel"].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [59]:
def mile(x):
  if x["fuel"] == "Patrol":
    return x["mileage"] / 80.43
  elif x["fuel"] == "Diesel":
    return x["mileage"] / 73.56
  elif x["fuel"] == "LPG":
    return x["mileage"] / 40.85
  else:
    return x["mileage"] / 44.23

In [62]:
data["mileage"] = data.apply(mile, axis=1)

In [63]:
data.drop("mileage_unit", axis=1, inplace=True)

In [65]:
data["torque"].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [66]:
data["torque"] = data["torque"].str.upper()

In [67]:
data["torque"].head()

0              190NM@ 2000RPM
1         250NM@ 1500-2500RPM
2       12.7@ 2,700(KGM@ RPM)
3    22.4 KGM AT 1750-2750RPM
4       11.5@ 4,500(KGM@ RPM)
Name: torque, dtype: object

In [70]:
"KGM" in data["torque"][0]

False

In [71]:
def torque_unit(x):
  if "NM" in str(x):
    return "Nm"
  elif "KGM" in str(x):
    return "Kgm"

In [72]:
data["torque_unit"] = data["torque"].apply(torque_unit)

In [73]:
data["torque_unit"].unique()

array(['Nm', 'Kgm', None], dtype=object)

In [80]:
data["torque"].head()

0              190NM@ 2000RPM
1         250NM@ 1500-2500RPM
2       12.7@ 2,700(KGM@ RPM)
3    22.4 KGM AT 1750-2750RPM
4       11.5@ 4,500(KGM@ RPM)
Name: torque, dtype: object

In [78]:
data[data["torque_unit"].isna()]["torque"].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [81]:
data["torque_unit"].fillna("Nm", inplace=True)

In [85]:
data[data["torque_unit"].isna()]["torque"].unique()

array([], dtype=object)

In [86]:
data["torque"].head()

0              190NM@ 2000RPM
1         250NM@ 1500-2500RPM
2       12.7@ 2,700(KGM@ RPM)
3    22.4 KGM AT 1750-2750RPM
4       11.5@ 4,500(KGM@ RPM)
Name: torque, dtype: object

In [88]:
tmp_str = "12.7@ 2,700(KGM@ RPM)"

In [97]:
for idx, val in enumerate(tmp_str):
  if val not in "0123456789.":
    cut = idx
    print(cut)
    break

4


In [108]:
def split_num(x):
  x_ = str(x)
  for idx, val in enumerate(x_):
    if val not in "0123456789.":
      cut = idx
      break
  return x_[:cut]

In [109]:
split_num(tmp_str)

'12.7'

In [110]:
data["torque"] = data["torque"].apply(split_num)

In [111]:
data["torque"]

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [112]:
dat["torque"] = data["torque"].astype("float64")

ValueError: could not convert string to float: ''

In [113]:
data["torque"] = data["torque"].replace("", np.nan)

In [115]:
data["torque"] = data["torque"].astype("float64")

In [116]:
data["torque_unit"].value_counts()

torque_unit
Nm     7624
Kgm     504
Name: count, dtype: int64

In [117]:
def torque_trans(x):
  if x["torque_unit"] == "Kgm":
    return x["torque"] * 9.8066
  else:
    return x["torque"]

In [123]:
torque_trans(data.loc[0, ["torque", "torque_unit"]])
# data.loc[0]

np.float64(190.0)

In [124]:
data["torque"] = data.apply(torque_trans, axis=1)

In [126]:
data.tail(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,torque_unit
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,0.320419,1396.0,70.0,140.0,5.0,Nm
8127,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,0.320419,1396.0,70.0,140.0,5.0,Nm


In [127]:
data.drop("torque_unit", axis=1, inplace=True)

In [130]:
data["name"].value_counts()

name
Maruti Swift Dzire VDI                             129
Maruti Alto 800 LXI                                 82
Maruti Alto LXi                                     71
BMW X4 M Sport X xDrive20d                          62
Maruti Swift VDI                                    61
                                                  ... 
Skoda Fabia 1.4 TDI Ambiente                         1
Mahindra Scorpio VLX 2WD AT BSIII                    1
Renault KWID Climber 1.0 AMT                         1
Mahindra XUV300 W8 Option Dual Tone Diesel BSIV      1
Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV         1
Name: count, Length: 2058, dtype: int64

In [131]:
data["name"] = data["name"].replace("Land", "Land Rover")

# 전처리 : 결측치

In [132]:
data.isna().mean() * 100

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          2.718996
engine           2.718996
max_power        2.657480
torque           2.731299
seats            2.718996
dtype: float64

In [133]:
data.dropna(inplace=True)

In [134]:
len(data)

7906

In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           7906 non-null   object 
 1   year           7906 non-null   int64  
 2   selling_price  7906 non-null   int64  
 3   km_driven      7906 non-null   int64  
 4   fuel           7906 non-null   object 
 5   seller_type    7906 non-null   object 
 6   transmission   7906 non-null   object 
 7   owner          7906 non-null   object 
 8   mileage        7906 non-null   float64
 9   engine         7906 non-null   float32
 10  max_power      7906 non-null   float32
 11  torque         7906 non-null   float64
 12  seats          7906 non-null   float64
dtypes: float32(2), float64(3), int64(3), object(5)
memory usage: 803.0+ KB


In [136]:
obj_list = []
for i in data.columns:
    if data[i].dtype == "object":
        obj_list.append(i)
obj_list

['name', 'fuel', 'seller_type', 'transmission', 'owner']

In [137]:
data = pd.get_dummies(data, columns=obj_list, drop_first=True)

In [138]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ambassador Classic 2000 DSZ AC PS,name_Ambassador Grand 1500 DSZ BSIII,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.000000,190.00000,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
1,2014,370000,120000,0.287384,1498.0,103.519997,250.00000,5.0,False,False,...,True,False,False,True,False,True,False,True,False,False
2,2006,158000,140000,0.400181,1497.0,78.000000,124.54382,5.0,False,False,...,False,False,True,True,False,True,False,False,False,True
3,2010,225000,127000,0.312670,1396.0,90.000000,219.66784,5.0,False,False,...,True,False,False,True,False,True,False,False,False,False
4,2007,130000,120000,0.364006,1298.0,88.199997,112.77590,5.0,False,False,...,False,False,True,True,False,True,False,False,False,False


In [139]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("selling_price", axis=1), data["selling_price"], test_size=0.2, random_state=50
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(6324, 1998) (1582, 1998)
(6324,) (1582,)


In [140]:
model = RandomForestRegressor(random_state=50)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [142]:
print("train rmse: ", round(root_mean_squared_error(y_train, train_pred), 2))
print("test rmse: ", round(root_mean_squared_error(y_test, test_pred),2 ))

train rmse:  55002.85
test rmse:  115864.59


# K-폴드 교차 검증

In [143]:
from sklearn.model_selection import KFold

In [145]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Columns: 1999 entries, year to owner_Third Owner
dtypes: bool(1991), float32(2), float64(3), int64(3)
memory usage: 15.5 MB


In [146]:
data.reset_index(drop=True, inplace=True)

In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7906 entries, 0 to 7905
Columns: 1999 entries, year to owner_Third Owner
dtypes: bool(1991), float32(2), float64(3), int64(3)
memory usage: 15.4 MB


In [150]:
kf = KFold(n_splits=5)
X = data.drop("selling_price", axis=1)
y = data["selling_price"]

In [152]:
for i, j in kf.split(X):
  print(i, j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [153]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y[train_index], y[test_index]

In [154]:
print(X_train.shape, X_test.shape)

(6325, 1998) (1581, 1998)


In [155]:
train_rmse_total = []
test_rmse_total = []

In [156]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestRegressor(random_state=50)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_rmse_total.append(root_mean_squared_error(y_train, train_pred))
    test_rmse_total.append(root_mean_squared_error(y_test, test_pred))

In [157]:
train_rmse_total

[50532.92942767605,
 55711.47280536497,
 53790.697984038095,
 54381.96940932169,
 54501.24982446165]

In [158]:
test_rmse_total

[173866.89158669254,
 135496.1191916217,
 125301.24567921956,
 151130.81632352594,
 142056.36112837086]

In [164]:
train_rmse_total = []
test_rmse_total = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestRegressor(n_estimators=300, max_depth=70, 
                                  min_samples_split=5, min_samples_leaf=1, n_jobs=-1,  random_state=50)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_rmse_total.append(root_mean_squared_error(y_train, train_pred))
    test_rmse_total.append(root_mean_squared_error(y_test, test_pred))
    print(train_rmse_total[len(train_rmse_total)-1])

57769.209003542295
68558.9134577012
62721.216716775125
64950.984882393204
64107.57630336222


In [161]:
test_rmse_total

[173866.89158669254,
 135496.1191916217,
 125301.24567921956,
 151130.81632352594,
 142056.36112837086,
 171739.74111250349,
 171739.74111250349,
 137296.59097606325,
 123987.6149926097,
 143623.36821695013,
 139073.40831448277]

# 렌덤포레스트 분류 모델 학습

In [193]:
from sklearn.ensemble import  RandomForestClassifier

In [168]:
file_url = (
    "https://media.githubusercontent.com/media/musthave-ML10/data_source/main/wine.csv"
)
data = pd.read_csv(file_url)
data.shape

(178, 14)

In [169]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       176 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    int64  
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          173 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    int64  
 13  class

In [170]:
data.dropna(inplace=True)

In [173]:
data.shape

(171, 14)

In [174]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       171 non-null    float64
 1   malic_acid                    171 non-null    float64
 2   ash                           171 non-null    float64
 3   alcalinity_of_ash             171 non-null    float64
 4   magnesium                     171 non-null    int64  
 5   total_phenols                 171 non-null    float64
 6   flavanoids                    171 non-null    float64
 7   nonflavanoid_phenols          171 non-null    float64
 8   proanthocyanins               171 non-null    float64
 9   color_intensity               171 non-null    float64
 10  hue                           171 non-null    float64
 11  od280/od315_of_diluted_wines  171 non-null    float64
 12  proline                       171 non-null    int64  
 13  class     

In [175]:
data.reset_index(drop=True, inplace=True)

In [177]:
data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
2,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
3,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0
4,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,0


In [190]:
kf = KFold(n_splits=5)
X = data.drop("class", axis=1)
y = data["class"]
train_score_total = []
test_score_total = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestClassifier(
        n_estimators=300,
        max_depth=5,
        min_samples_split=5,
        min_samples_leaf=1,
        n_jobs=-1,
        random_state=50,
    )
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_score_total.append(accuracy_score(y_train, train_pred))
    test_score_total.append(accuracy_score(y_test, test_pred))
    print(
        round(train_score_total[len(train_score_total) - 1], 3),
        " : ",
        round(test_score_total[len(test_score_total) - 1], 3),
    )

1.0  :  0.914
1.0  :  0.971
1.0  :  0.941
1.0  :  1.0
1.0  :  0.971


In [187]:
rs_scaler = RobustScaler()
data_scaled = rs_scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,0.897338,-0.103226,0.205882,-0.906977,1.487179,0.467290,0.536023,-0.324324,1.013889,0.280936,0.202899,0.915663,0.811983,-0.5
1,0.083650,0.316129,0.911765,-0.209302,0.153846,0.467290,0.639769,-0.216216,1.736111,0.294314,0.173913,0.313253,1.059917,-0.5
2,1.003802,0.051613,0.411765,-0.627907,0.769231,1.448598,0.783862,-0.540541,0.861111,1.003344,-0.318841,0.538153,1.669421,-0.5
3,0.144487,0.464516,1.500000,0.348837,1.025641,0.467290,0.322767,0.270270,0.361111,-0.160535,0.202899,0.120482,0.130165,-0.5
4,0.874525,-0.070968,0.264706,-1.000000,0.717949,0.906542,0.726225,0.000000,0.569444,0.652174,0.231884,0.056225,1.607438,-0.5


In [ ]:
kf = KFold(n_splits=5)
X = data_scaled.drop("class", axis=1)
y = data["class"]
train_score_total = []
test_score_total = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = RandomForestClassifier(
        n_estimators=300,
        max_depth=3,
        min_samples_split=5,
        min_samples_leaf=1,
        n_jobs=-1,
        random_state=50,
    )
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_score_total.append(accuracy_score(y_train, train_pred))
    test_score_total.append(accuracy_score(y_test, test_pred))
    print(
        round(train_score_total[len(train_score_total) - 1], 3),
        " : ",
        round(test_score_total[len(test_score_total) - 1], 3),
    )

1.0  :  0.857
0.993  :  0.971
1.0  :  0.941
1.0  :  1.0
1.0  :  0.971


# AdaBoostClassifier

from sklearn.ensemble import AdaBoostClassifier  
from sklearn.ensemble import BaggingClassifier  
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.ensemble import HistGradientBoostingClassifier  
from sklearn.ensemble import VotingClassifier  

In [196]:
help(AdaBoostClassifier)

Help on class AdaBoostClassifier in module sklearn.ensemble._weight_boosting:

class AdaBoostClassifier(sklearn.utils._metadata_requests._RoutingNotSupportedMixin, sklearn.base.ClassifierMixin, BaseWeightBoosting)
 |  AdaBoostClassifier(estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='deprecated', random_state=None)
 |  
 |  An AdaBoost classifier.
 |  
 |  An AdaBoost [1]_ classifier is a meta-estimator that begins by fitting a
 |  classifier on the original dataset and then fits additional copies of the
 |  classifier on the same dataset but where the weights of incorrectly
 |  classified instances are adjusted such that subsequent classifiers focus
 |  more on difficult cases.
 |  
 |  This class implements the algorithm based on [2]_.
 |  
 |  Read more in the :ref:`User Guide <adaboost>`.
 |  
 |  .. versionadded:: 0.14
 |  
 |  Parameters
 |  ----------
 |  estimator : object, default=None
 |      The base estimator from which the boosted ensemble is built.
 |   

In [206]:
from sklearn.ensemble import AdaBoostClassifier

kf = KFold(n_splits=5)
X = data_scaled.drop("class", axis=1)
y = data["class"]
train_score_total = []
test_score_total = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = AdaBoostClassifier(
        n_estimators=300,
        learning_rate=1.0,
        random_state=50,
    )
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_score_total.append(accuracy_score(y_train, train_pred))
    test_score_total.append(accuracy_score(y_test, test_pred))
    print(
        round(train_score_total[len(train_score_total) - 1], 3),
        " : ",
        round(test_score_total[len(test_score_total) - 1], 3),
    )

1.0  :  0.943
1.0  :  0.882
1.0  :  1.0
1.0  :  0.824
1.0  :  0.853


# GradientBoostingClassifier

In [207]:
help(GradientBoostingClassifier)

Help on class GradientBoostingClassifier in module sklearn.ensemble._gb:

class GradientBoostingClassifier(sklearn.base.ClassifierMixin, BaseGradientBoosting)
 |  GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for classification.
 |  
 |  This algorithm builds an additive model in a forward stage-wise fashion; it
 |  allows for the optimization of arbitrary differentiable loss functions. In
 |  each stage ``n_classes_`` regression trees are fit on the negative gradient
 |  of the loss function, e.g. binary or multiclass log loss. Binary
 |  classification is a special case where only a single regression

In [208]:
from sklearn.ensemble import GradientBoostingClassifier

kf = KFold(n_splits=5)
X = data_scaled.drop("class", axis=1)
y = data["class"]
train_score_total = []
test_score_total = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = GradientBoostingClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=5,
        random_state=50,
    )
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_score_total.append(accuracy_score(y_train, train_pred))
    test_score_total.append(accuracy_score(y_test, test_pred))
    print(
        round(train_score_total[len(train_score_total) - 1], 3),
        " : ",
        round(test_score_total[len(test_score_total) - 1], 3),
    )

1.0  :  0.943
1.0  :  0.912
1.0  :  0.735
1.0  :  0.912
1.0  :  0.882
